In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
housing = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/house_price_data.csv")


In [ ]:
housing.head()

**Data Prepration**

In [ ]:
housing=housing.drop(columns=["Unnamed: 0", 'property_id', 'location_id','page_url','province_name', 'latitude','longitude','date_added','agency','agent'], axis=1)

In [ ]:
housing.head(3)
print(housing.shape)

In [ ]:
housing=housing[housing['purpose']=='For Sale']
housing.shape

In [ ]:
housing = housing[housing['city'] == 'Islamabad']

In [ ]:
housing.shape

In [ ]:
housing.head(3)

In [ ]:
housing = housing[housing['property_type'] == 'House']

In [ ]:
housing['price'] = housing['price'].apply(lambda x: x/100000)

In [ ]:
housing = housing[housing['price'] < 4000]

In [ ]:
housing.head(3)

In [ ]:
housing=housing.drop(columns=['property_type', 'city', 'purpose'], axis=1)

In [ ]:
housing.head(3)
print(housing.shape)

In [ ]:
print(housing['location'].value_counts())

In [ ]:
housing['location'] = housing['location'].apply(lambda x: x.strip())
location_count = housing['location'].value_counts()
print(type(location_count))

In [ ]:
location_count_less_10 = location_count[location_count <= 10]

In [ ]:
housing['location'] = housing['location'].apply(lambda x: 'other' if x in location_count_less_10 else x)

In [ ]:

housing['location'].nunique()
print(housing.isnull().sum())

**Feature Engineering**


In [ ]:
housing.head(2)
housing.shape

In [ ]:
X= housing.iloc[:,1:5]
y= housing.iloc[:,0]

In [ ]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split( X, y, train_size=0.8, random_state=0)


In [ ]:
X_train.shape[0]
X_test.shape[0]
X=X_train.shape[0]+X_test.shape[0]
print(X)

In [ ]:
housing.head(3)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler



# First column transformer (first pipeline element)
ct1= ColumnTransformer(transformers=[
    ("ohe_location", OneHotEncoder(sparse_output=False, drop="first", handle_unknown="ignore", dtype=np.int32),['location'])
], remainder="passthrough")


#Second column transformer (second pipeline element)
ct2= ColumnTransformer(transformers=[
    ("ss_location", StandardScaler(),['location'])
], remainder="passthrough")


random_forest = DecisionTreeRegressor(max_depth=300, splitter= "best", min_impurity_decrease= 0.005,random_state=0)

pipe = make_pipeline(ct1,ct2 ,random_forest)




In [ ]:
# First column transformer (first pipeline element)
ct1= ColumnTransformer(transformers=[
    ("ohe_location", OneHotEncoder(sparse_output=False, drop="first", handle_unknown="ignore", dtype=np.int32),['location'])
], remainder="passthrough")

In [ ]:
#Second column transformer (second pipeline element)
ct2= ColumnTransformer(transformers=[
    ("ss_location", StandardScaler(),['location'])
], remainder="passthrough")

In [ ]:
random_forest = DecisionTreeRegressor(max_depth=300, splitter= "best", min_impurity_decrease= 0.005,random_state=0)

In [ ]:
pipe = make_pipeline(ct1,ct2 ,random_forest)

In [ ]:
from sklearn import set_config
set_config(display='diagram')

In [ ]:
X_train = pd.DataFrame(X_train, columns=['location','baths', 'bedrooms', 'Total_Area'])

y_train= pd.DataFrame(y_train, columns=['price'])
pipe.fit(X_train, y_train)

In [ ]:
y_pred= pipe.predict(X_test)

In [ ]:
r2_score(y_test, y_pred)